In [21]:
# 데이터 불러오기
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

df = pd.read_csv("서울시 문화공간 정보.csv", encoding="EUC-KR", encoding_errors="replace", low_memory=False)
print(df.head())
print(df.info())

   번호    주제분류           문화시설명                       주소         위도          경도  \
0   1      기타  DDP(동대문디자인플라자)         서울특별시 중구 을지로 281  37.567357  127.009780   
1   2     공연장            우란2경    서울특별시 성동구 연무장7길 11 2층  37.543790  127.054417   
2   3     도서관      구립은평뉴타운도서관    서울특별시 은평구 진관2로 111-51  37.637230  126.933118   
3   4     공연장            씨어터쿰     서울특별시 종로구 창경궁로 253-7  37.584423  126.999528   
4   5  문화예술회관        노원문화예술회관  서울 노원구 중계로 181 노원문화예술회관  37.650237  127.080267   

             전화번호          팩스번호  \
0    02-2153-0000  02-2153-0010   
1   070-4244-3812           NaN   
2  (02) 6341-6400           NaN   
3    02-6085-0015           NaN   
4    02-2289-3425           NaN   

                                                홈페이지 관람시간  ... 기타사항  \
0                               http://www.ddp.or.kr  NaN  ...  NaN   
1          https://www.wooranfdn.org/about/space.jsp  NaN  ...  NaN   
2                            http://www.enlib.or.kr/  NaN  ...  NaN   
3  https://www.dae

In [23]:
new_df = df[["주제분류","문화시설명","위도","경도"]]

In [25]:
geojson_file = "hangjeongdong_서울특별시.geojson"
gdf = gpd.read_file(geojson_file)

def find_administrative_dong(latitude, longitude, gdf):
    point = Point(longitude, latitude)
    result = gdf[gdf.contains(point)]
    return result.iloc[0]["adm_nm"] if not result.empty else "해당 좌표는 어떤 행정동에도 속하지 않습니다."

new_df["행정동"] = new_df.apply(lambda x: find_administrative_dong(x["위도"], x["경도"], gdf), axis=1)

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_23092\1534519273.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["행정동"] = new_df.apply(lambda x: find_administrative_dong(x["위도"], x["경도"], gdf), axis=1)


In [32]:
final_data = pd.pivot_table(new_df, index="주제분류", columns="행정동", aggfunc="size", fill_value=0)

In [34]:
final_data

행정동,서울특별시 강남구 개포2동,서울특별시 강남구 논현1동,서울특별시 강남구 논현2동,서울특별시 강남구 대치2동,서울특별시 강남구 대치4동,서울특별시 강남구 도곡1동,서울특별시 강남구 도곡2동,서울특별시 강남구 삼성1동,서울특별시 강남구 삼성2동,서울특별시 강남구 세곡동,...,서울특별시 중구 황학동,서울특별시 중구 회현동,서울특별시 중랑구 망우본동,서울특별시 중랑구 면목4동,서울특별시 중랑구 면목7동,서울특별시 중랑구 면목본동,서울특별시 중랑구 묵1동,서울특별시 중랑구 상봉1동,서울특별시 중랑구 상봉2동,서울특별시 중랑구 신내2동
주제분류,,,,,,,,,,,,,,,,,,,,,
공연장,0,2,1,3,0,0,1,2,1,0,...,0,0,0,0,0,0,0,0,0,0
기타,0,2,3,2,0,0,0,1,1,0,...,1,4,0,1,2,0,0,0,0,1
도서관,1,0,1,2,1,1,0,1,1,3,...,0,0,1,0,0,1,2,0,1,0
문화예술회관,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
문화원,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
미술관/갤러리,0,0,0,1,1,0,0,3,0,0,...,0,0,0,0,0,0,0,1,0,0
박물관/기념관,1,0,1,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
